In [1]:
print("Hello World")

Hello World


In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Manas\\OneDrive\\Desktop\\AI ML\\medical-chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
def loader_file(file):
    loader = DirectoryLoader(file,glob = "*.pdf",loader_cls=PyPDFLoader)
    document = loader.load()
    return document

In [10]:
extracted_information = loader_file(file = 'Data')

In [12]:
def textsplit(data):
    text_split = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunk = text_split.split_documents(data)
    return text_chunk

In [13]:
text_chunks = textsplit(extracted_information)
print("Total Length of the Textual Chunks : ", len(text_chunks))

Total Length of the Textual Chunks :  5859


In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [21]:
embeddings = download_hugging_face_embeddings()

c:\Users\Manas\OneDrive\Desktop\AI ML\medical-chatbot\myvenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Manas\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not

In [22]:
query_result = embeddings.embed_query("Hello World")
print("Length of the embedded query is : ", len(query_result))

Length of the embedded query is :  384


In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [61]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [47]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os


index_name = "medicalbot"

pc = Pinecone(api_key=PINECONE_API_KEY)

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        dimension = 384,
        metric = "cosine",
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [ ]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [35]:
from langchain_pinecone import PineconeVectorStore
search = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name = index_name,
    embedding=embeddings
)

In [36]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [37]:
docsearch

In [111]:
retrieve = docsearch.as_retriever(search_type = "similarity",search={"k":3})

In [39]:
retrived_docs = retrieve.invoke("What is Acne?")

In [40]:
retrived_docs

[Document(id='c66769b9-e38e-4af2-b001-57cbc102dbb6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='237766eb-4f7d-4a0c-9912-d5b55d33adac', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [109]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document, BaseRetriever

# Setup local FLAN-T5 text2text generation pipeline
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_new_tokens=150,
    temperature=0.0,
    device=-1  # CPU. Change to 0 for GPU if available
)

llm = HuggingFacePipeline(pipeline=pipe)

# Define the system prompt to guide the model's behavior
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

qa_chain = create_stuff_documents_chain(llm, prompt)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [112]:
rag_chain = create_retrieval_chain(retrieve, qa_chain)


In [117]:
query = "What is stats?"


In [118]:
response = rag_chain.invoke({"input": query})


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [119]:
print(response["answer"])


The yearly rate for occurrence of SUD in people less than 35 years of age is less than seven incidents per 100,000. Of all SUD cases, only about 8% are exercise related.
